# Use transfer-learning with huggingface-transformers library

## Load data and make it accessible for huggingface environment

In [57]:
%load_ext autoreload
%autoreload 2
from transformers import (AutoTokenizer, 
                          TFAutoModel,
                          TFAutoModelForSequenceClassification
                          )
from datasets import Dataset, DatasetDict, load_from_disk # to use huggingface datasets
from detector.utils import load_data, divide_frame
import tensorflow as tf
import torch
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, PredefinedSplit

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
data = load_data()

In [4]:
train, val, test = (data["train"].reset_index(drop=True), 
                    data["valid"].reset_index(drop=True), 
                    data["test"].reset_index(drop=True))

def remove_newline(text: str) -> str:
    return text.replace("\n", " ")


for df in [train, val, test]:
    df["text"] = df["text"].apply(remove_newline)
    df["text_length"] = df['text'].apply(len)

In [5]:
train_small, train_medium, train_long = divide_frame(train)

In [6]:
for df in [train_small, train_medium, train_long]:
    print(df.shape[0])

train_sample = pd.concat([train_small.sample(10_000, random_state=1), 
                          train_medium.sample(20_000, random_state=1),
                          train_long.sample(10_000, random_state=1)]).reset_index(drop=True)

124936
250122
124942


In [7]:
train_sample

,text,AI,text_length
0,"""I didn't even know what a baby looked like un...",1,264
1,"In this video, watch as an electric eel can cl...",1,1043
2,"IOTA is divisible, fungible, durable , portabl...",0,529
3,× Report Chanel Preston And Ryan Ryans Just A ...,0,330
4,The best-selling film of 2012 starring Christi...,1,987
...,...,...,...
39995,"BANGUI, Central African Republic (AP) — A day ...",1,4709
39996,So what world view is under assault by populis...,0,5219
39997,This is a list of the most frequently asked qu...,1,5083
39998,Gerardo Mora/GettyImages. Mitt Romney has a p...,0,4951


In [8]:
ds_train = Dataset.from_pandas(train_sample, split="train")
ds_val = Dataset.from_pandas(val, split="valid")
ds_test = Dataset.from_pandas(test, split="test")

In [9]:
# pack datasets into a dictionary to tokenize them in parallel
ds_dict = DatasetDict({"train": ds_train, "valid": ds_val, "test": ds_test})

In [10]:
ds_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'AI', 'text_length'],
        num_rows: 40000
    })
    valid: Dataset({
        features: ['text', 'AI', 'text_length'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'AI', 'text_length'],
        num_rows: 10000
    })
})

## Create Tokenizer suitable for the model

In [11]:
def model_checkpoint(name: str="distilbert", 
                     large: bool=True,
                     uncased: bool=True) -> str:
    model_ckpt = f'{name}-large' if large else f'{name}-base'
    return f'{model_ckpt}-uncased' if uncased else model_ckpt

model_ckpt = model_checkpoint(large=False)
# define the tokenizer the model was trained with
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model_ckpt

'distilbert-base-uncased'

In [12]:
tokenizer("this is a test!", return_tensors="tf")

2023-06-19 18:33:59.261281: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-06-19 18:33:59.261348: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-06-19 18:33:59.261359: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-06-19 18:33:59.261544: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-06-19 18:33:59.261730: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


{'input_ids': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[ 101, 2023, 2003, 1037, 3231,  999,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 7), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [13]:
tokenizer.model_input_names

['input_ids', 'attention_mask']

In [14]:
# define a tokenize function that tokenizes the text in batches
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [15]:
ds_encoded = ds_dict.map(tokenize, batched=True, batch_size=10_000)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
ds_encoded["train"].column_names

['text', 'AI', 'text_length', 'input_ids', 'attention_mask']

## Option 1: Use pre-trained model as feature extractor

### Extracting last hidden layer of a BERT model

For this approach the model-weights of our RoBERTA model are frozen and provide features for a classifier 

In [17]:
model = TFAutoModel.from_pretrained(model_ckpt, from_pt=True) # load the model from the checkpoint

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [18]:
text = "this is a test"
inputs = tokenizer(text, return_tensors="tf")
outputs = model(**inputs)
outputs

TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 6, 768), dtype=float32, numpy=
array([[[-0.15651302, -0.18619634,  0.05277671, ..., -0.11881141,
          0.06620619,  0.5470155 ],
        [-0.35751384, -0.64835554, -0.06178989, ..., -0.30401945,
          0.35076857,  0.5220684 ],
        [-0.27718446, -0.44594443,  0.18184225, ..., -0.09477933,
         -0.00757482,  0.99582803],
        [-0.2840852 , -0.39167672,  0.37525553, ..., -0.21505716,
         -0.11725152,  1.0526478 ],
        [ 0.26608223, -0.50936383, -0.3180131 , ..., -0.42029774,
          0.01444196, -0.21489492],
        [ 0.9440609 ,  0.01117296, -0.47139424, ...,  0.1439469 ,
         -0.7287833 , -0.16194938]]], dtype=float32)>, hidden_states=None, attentions=None)

In [19]:
outputs.last_hidden_state.shape #output [batch_size, n_tokens, hidden_dim]

TensorShape([1, 6, 768])

In [20]:
# for classification it is common practice use hidden state associated to start 
# of sequence token
outputs.last_hidden_state[:, 0].shape 

TensorShape([1, 768])

In [21]:
# extract last hidden state for whole dataset
def extract_hidden_states(batch):
    inputs = {k: v for k,v in batch.items() if k in tokenizer.model_input_names}
    last_hidden_state = model(**inputs).last_hidden_state
    return {"hidden_state": last_hidden_state[:, 0].numpy()}

In [22]:
ds_encoded.set_format("tensorflow", columns=["input_ids", "attention_mask", "AI"])

In [23]:
extract_hidden_states(ds_encoded["train"][:2])

{'hidden_state': array([[ 0.17704922,  0.00966936, -0.16979757, ..., -0.05526693,
          0.26920223,  0.255841  ],
        [-0.20750733, -0.3828969 , -0.23853931, ..., -0.06183667,
          0.51362616,  0.57837814]], dtype=float32)}

In [24]:
ds_hidden = ds_encoded.map(extract_hidden_states, batched=True, batch_size=50)

/Users/michaelfiedler/.pyenv/versions/3.10.6/envs/AI_written_text_identifier/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [25]:
ds_hidden

DatasetDict({
    train: Dataset({
        features: ['text', 'AI', 'text_length', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 40000
    })
    valid: Dataset({
        features: ['text', 'AI', 'text_length', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'AI', 'text_length', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 10000
    })
})

In [56]:
ds_hidden.save_to_disk("hidden_states")

In [58]:
ds_hidden = load_from_disk("hidden_states")

-----

### Preparing data to train model

We are now using these hidden states to train a relatively simple classifier to predict if text is AI written or not.

In [29]:
X_train = np.array(ds_hidden["train"]["hidden_state"])
X_val = np.array(ds_hidden["valid"]["hidden_state"])
X_test = np.array(ds_hidden["test"]["hidden_state"])

y_train = np.array(ds_hidden["train"]["AI"])
y_val = np.array(ds_hidden["valid"]["AI"])
y_test = np.array(ds_hidden["test"]["AI"])

In [31]:
X_train.shape, y_train.shape

((40000, 768), (40000,))

In [37]:
X_search = np.vstack((X_train, X_val))
y_search = np.hstack((y_train, y_val))

In [38]:
X_search.shape, y_search.shape
split = PredefinedSplit([-1]*X_train.shape[0]+[0]*X_val.shape[0])

((50000, 768), (50000,))

### Logistic Regression

In [36]:
lr_clf = LogisticRegression(max_iter=3000)

params = {"C":[2**k for k in range(-2, 5)]}

search = GridSearchCV(lr_clf,
                      param_grid=params,
                      n_jobs=-1,
                      cv = split,
                      scoring="accuracy")

In [39]:
search.fit(X_search, y_search)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=LogisticRegression(max_iter=3000), n_jobs=-1,
             param_grid={'C': [0.25, 0.5, 1, 2, 4, 8, 16]}, scoring='accuracy')

In [42]:
lr_clf_best = search.best_estimator_

In [44]:
lr_clf_best.fit(X_train, y_train)
lr_clf_best.score(X_test, y_test)

0.871

### SVM Classifier

In [45]:
from sklearn.svm import SVC

In [60]:
svc = SVC(random_state=1)

params_svc = {
    'C': [0.5],
    #'degree': 3,
    'kernel': ['linear']
 }

search_svc = GridSearchCV(
    svc,
    param_grid=params_svc,
    n_jobs=-1,
    cv=split,
    scoring="accuracy"
)

In [61]:
search_svc.fit(X_search, y_search)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=SVC(random_state=1), n_jobs=-1,
             param_grid={'C': [0.5], 'kernel': ['linear']}, scoring='accuracy')

In [62]:
search_svc.best_score_

0.8781

In [63]:
svc_best = search_svc.best_estimator_

In [64]:
svc_best.fit(X_train, y_train)

SVC(C=0.5, kernel='linear', random_state=1)

In [65]:
svc_best.score(X_test, y_test)

0.8725